In [5]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\mariu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


**PRUEBA AISLADA DE CONEXION A SQL SERVER**

In [1]:
import pyodbc

server = 'MARIUGENIAIT\SQLEXPRESS'
database = 'master'
driver = '{ODBC Driver 17 for SQL Server}'

try:
    conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a SQL Server")
except pyodbc.Error as ex:
    print(f"Error de conexión: {ex}")


Conexión exitosa a SQL Server


**CREACION DE BASE DE DATOS**

In [2]:
import subprocess

# Datos de conexión a SQL Server (Autenticación de Windows integrada)
server = 'MARIUGENIAIT\SQLEXPRESS'  # Reemplaza con el nombre de tu servidor
database = 'InventarioLicores'  # Nombre de la base de datos

try:
    # Crear la base de datos usando sqlcmd (proceso aislado)
    sqlcmd_command = [
        "sqlcmd",
        "-S", server,
        "-E",  # Autenticación integrada de Windows
        "-Q", f"IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = '{database}') CREATE DATABASE {database}"
    ]
    subprocess.run(sqlcmd_command, check=True)
    print(f"Base de datos '{database}' creada o ya existente.")

except subprocess.CalledProcessError as e:
    print(f"Error al crear la base de datos con sqlcmd: {e}")
except Exception as ex:
    print(f"Error general: {ex}")

Base de datos 'InventarioLicores' creada o ya existente.


**CREACION DE LAS TABLAS Y SUS RELACIONES E INDICES**

In [3]:
import subprocess

# Datos de conexión a SQL Server (Autenticación de Windows integrada)
server = 'MARIUGENIAIT\SQLEXPRESS'  # Reemplaza con el nombre de tu servidor
database = 'InventarioLicores'  # Nombre de la base de datos

# Comandos SQL para crear las tablas, claves y relaciones
sql_commands = """
    USE InventarioLicores;

    -- Crear la tabla de Productos
    CREATE TABLE Productos (
        productoId INT PRIMARY KEY,
        nombreProducto VARCHAR(100),
        precio DECIMAL(10,2),
        tamanio VARCHAR(100),
        volumen INT,
        clasificacion INT,
        costo DECIMAL(10, 2),
        proveedorId INT,
        nombreProveedor VARCHAR(100)
    );

    -- Crear la tabla de Compras
    CREATE TABLE Compras (
        compraId INT PRIMARY KEY,
        proveedorId INT,
        nombreProveedor VARCHAR(100),
        fechaFactura DATE,
        fechaCompra DATE,
        fechaPago DATE,
        cantidad INT,
        costoTotal DECIMAL(18, 2),
        costoEnvio DECIMAL(18, 2)
    );

    -- Crear la tabla de InventarioFinal
    CREATE TABLE InventarioFinal (
        inventarioFinId INT PRIMARY KEY,
        tienda INT,
        ciudad VARCHAR(100),
        productoId INT,
        nombreProducto VARCHAR(100),
        tamanio VARCHAR(100),
        cantidad INT,
        precio DECIMAL(18, 2),
        fechaFin DATE,
        FOREIGN KEY (productoId) REFERENCES Productos(productoId)
    );

    -- Crear la tabla de InventarioInicial
    CREATE TABLE InventarioInicial (
        inventarioIniId INT PRIMARY KEY,
        tienda INT,
        ciudad VARCHAR(100),
        productoId INT,
        nombreProducto CHAR(100),
        tamanio VARCHAR(100),
        cantidad INT,
        precio DECIMAL(18, 2),
        fechaInicio DATE,
        FOREIGN KEY (productoId) REFERENCES Productos(productoId)
    );

    -- Crear la tabla de Ventas
    CREATE TABLE Ventas (
        ventasId INT PRIMARY KEY,
        inventarioIniId INT,
        tienda INT,
        productoId INT,
        nombreProducto VARCHAR(100),
        tamanio VARCHAR(100),
        cantidadVendida INT,
        totalVenta DECIMAL(18, 2),
        precioUnitario DECIMAL(18, 2),
        volumen INT,
        clasificacion INT,
        impuestos DECIMAL(9, 2),
        proveedorId INT,
        nombreProveedor VARCHAR(100),
        FOREIGN KEY (inventarioIniId) REFERENCES InventarioInicial(inventarioIniId),
        FOREIGN KEY (productoId) REFERENCES Productos(productoId)
    );

    -- Crear la tabla de DetalleCompra
    CREATE TABLE DetalleCompra (
        detalleCompraId INT PRIMARY KEY,
        inventarioIniId INT,
        tienda INT,
        productoId INT,
        nombreProducto VARCHAR(100),
        tamanio VARCHAR(100),
        proveedorId INT,
        nombreProveedor VARCHAR(100),
        compraId INT,
        fechaCompra DATE,
        fechaIngreso DATE,
        fechaFactura DATE,
        fechaPago DATE,
        costoUnitario DECIMAL(18, 2),
        cantidad INT,
        costoTotal DECIMAL(9, 2),
        clasificacion INT,
        FOREIGN KEY (inventarioIniId) REFERENCES InventarioInicial(inventarioIniId),
        FOREIGN KEY (productoId) REFERENCES Productos(productoId),
        FOREIGN KEY (compraId) REFERENCES Compras(compraId)
    );

    -- Crear los índices
    CREATE INDEX IX_Productos_NombreProducto
    ON Productos (nombreProducto);

    CREATE INDEX IX_Compras_FechaCompra
    ON Compras (fechaCompra);

    CREATE INDEX IX_DetalleCompra_ProductoID
    ON DetalleCompra (productoId);

    CREATE INDEX IX_Ventas_InventarioID
    ON Ventas (inventarioIniId);
"""

# Ejecutar el comando SQL con sqlcmd
try:
    # Crear las tablas usando sqlcmd
    sqlcmd_command = [
        "sqlcmd",
        "-S", server,
        "-E",  # Autenticación integrada de Windows
        "-Q", sql_commands
    ]
    subprocess.run(sqlcmd_command, check=True)
    print(f"Tablas, relaciones, claves PK y FK, e índices creados exitosamente en la base de datos '{database}'.")

except subprocess.CalledProcessError as e:
    print(f"Error al crear las tablas con sqlcmd: {e}")
except Exception as ex:
    print(f"Error general: {ex}")


Tablas, relaciones, claves PK y FK, e índices creados exitosamente en la base de datos 'InventarioLicores'.
